In [7]:
# import os
# import glob
# import numpy as np
# from scipy.io import loadmat
# import seaborn as sns
# import pandas as pd
# import matplotlib.pyplot as plt
# import timecorr as tc
# import math
# import scipy.spatial.distance as sd
# from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
# corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'
# factors = 100

# if factors == 100:
#     pieman_name = '../../data/pieman_ica100.mat'
# else:
#     pieman_name = '../../data/pieman_data.mat'
    

# pieman_data = loadmat(pieman_name)  

# pieman_conds = ['word']
# data = []
# conds = []
# for c in pieman_conds:
#     if c == 'paragraph':
#         if factors == 700:
#             next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
#         else:
#             next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
#     else:
#         next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
#     data.extend(next_data)
#     conds.extend([c]*len(next_data))

# all_data = np.array(data)
# conds = np.array(conds)
# cfun = isfc
# rfun = 'eigenvector_centrality'
# width = 10
# wp = 'gaussian'
# cond = 'intact'
# level = 1

# gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

# weights_paramter = eval(wp)


# weights_fun=weights_paramter['weights']
# weights_params=weights_paramter['params']
# combine = corrmean_combine  
# group_chunks = 2


# ## just for word group 2
# #### RUN FOR 700 FACTORS ON CLUSTER
# for c in pieman_conds:
#     g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
#     for i in range(2):
#         data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#         lev = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=None,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#         corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
#         np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)




In [ ]:
def order_up(data, level=0, cfun=None, weights_fun=None, weights_params=None, combine=None, rfun=None):

    from .timecorr import timecorr

    if rfun is None:
        rfun = [None] * np.shape(level)[0]

    if level == 0:

        smooth = np.asarray(timecorr([x for x in data], cfun=None,
                                             rfun=rfun[level], combine=combine[level], weights_function=weights_fun,
                                             weights_params=weights_params))
        raw = mean_combine([x for x in data])
        
    else:
        smooth = np.asarray(timecorr(data, cfun=cfun[level], rfun=rfun[level], combine=combine[level],
                                                 weights_function=weights_fun, weights_params=weights_params))
        raw = np.asarray(timecorr(data, cfun=cfun[level], rfun=rfun[level], combine=null_combine,
                                              weights_function=eye_weights, weights_params=eye_params))


    return smooth, raw

In [ ]:

def reduce_wrap(data, dims=10, level=0, rfun=None):

    if not level == 0:

        all_smooth = list(data[0][np.newaxis, :, :]) + list(data[1][np.newaxis, :, :])
        all_raw = list(data[2][np.newaxis, :, :]) + list(data[3][np.newaxis, :, :])

        all_smooth_reduced = reduce(all_smooth, rfun=rfun[level])
        all_raw_reduced = reduce(all_raw, rfun=rfun[level])


        return all_smooth_reduced[0], all_smooth_reduced[1], all_raw_reduced[0], all_raw_reduced[1]

    else:

        return data[0], data[1], data[2], data[3]

In [26]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce

factors = 100
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']

data = []
conds = []


for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))


all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 50
smooth = 'laplace'
raw = 'delta'

#results_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results/'

results_dir = '/Users/lucyowen/Desktop/timecorr_env/timecorr_paper/pieman/results/'

corrsdir = os.path.join(results_dir, wp + '_' + str(50) + '_orderedup_corrs')

if not os.path.exists(corrsdir):
    os.makedirs(corrsdir)
    
laplace = {'name': 'Laplace', 'weights': tc.laplace_weights, 'params': {'scale': width}}
delta = {'name': '$\delta$', 'weights': tc.eye_weights, 'params': tc.eye_params}
gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}
mexican_hat = {'name': 'Mexican hat', 'weights': tc.mexican_hat_weights, 'params': {'sigma': width}}

smooth_parameter = eval(smooth)
raw_parameter = eval(raw)

smooth_fun=smooth_parameter['weights']
smooth_params=smooth_parameter['params']
raw_fun=raw_parameter['weights']
raw_params=raw_parameter['params']

combine = corrmean_combine  

levels = range(3)

print(levels)

for c in pieman_conds:

    for lev in levels:
        
        if os.path.exists(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy')):
            data = np.load(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy'))
            print(str(lev) + '_data loaded')
        else:
            
            if lev == 0:
                lev_data = all_data[conds == c]
            else:
                lev_data = data_r
                
            print('lev_data: ' + str(lev_data[0][0]))
            
            data = np.asarray(tc.timecorr([x for x in lev_data], cfun=isfc, rfun=None,
                                          weights_function=smooth_fun, weights_params=smooth_params))
            
            print('data: ' + str(data[0][0]))
            print(str(lev) + '_smooth data calculated')
            
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data, axis=2)), axis=2))
            
            print(str(lev) + '_corrs calculated')
            
            np.save(os.path.join(corrsdir, 'lev_'+ str(lev) +'_'+ c +'.npy'), corr)
            
            data = np.asarray(tc.timecorr([x for x in lev_data], cfun=isfc, rfun=None,
                                          weights_function=raw_fun, weights_params=raw_params))
            
            print(str(lev) + '_raw data calculated')
            
            np.save(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy'), data)
            

        if os.path.exists(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy')):
            data_r = np.load(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy'))
            print(str(lev) + '_reduced data loaded')
        else:
            data_r = np.asarray(reduce([x for x in data], rfun=rfun))
            print(str(lev) + '_reduced data calculated')
            np.save(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy'), data_r)

            print('data_r: ' + str(data_r[0][0]))
        del data
        

range(0, 3)
0_data loaded
0_reduced data loaded
lev_data: [0.14302537 0.00982672 0.08145851 0.05860905 0.07720654 0.13107708
 0.08700501 0.12014306 0.02272707 0.12851244 0.09639177 0.13539257
 0.01057133 0.13790108 0.10585685 0.1308951  0.00367713 0.12067527
 0.09668716 0.12913936 0.128724   0.07483774 0.0746319  0.02725164
 0.08344455 0.12127144 0.08733651 0.14921866 0.02892407 0.12531941
 0.00920434 0.10757915 0.00811552 0.03756734 0.11342522 0.07662683
 0.13405174 0.05758245 0.02685659 0.12033433 0.06650679 0.05281363
 0.1424802  0.10382324 0.0140855  0.04675881 0.09704961 0.1055391
 0.05488077 0.13526464 0.02879268 0.00695268 0.08356988 0.02071912
 0.03402362 0.13897895 0.1128056  0.11573054 0.06335576 0.07580266
 0.13779102 0.1006125  0.13705344 0.12579201 0.04377354 0.11815308
 0.10788544 0.13513238 0.12071263 0.11163909 0.12464418 0.13629674
 0.09941703 0.13384385 0.07131796 0.12588853 0.12783232 0.0322023
 0.06038959 0.11707526 0.08449785 0.11743815 0.12465436 0.12224591
 0.121

In [17]:
data_0 = np.load(os.path.join(corrsdir, 'lev_0_rest.npy'))

In [18]:
data_0

array([[ 0.00472924,  0.00292534, -0.00800761, ..., -0.03615471,
         0.00085978,  0.02894596],
       [ 0.00473935,  0.00292719, -0.00800845, ..., -0.0361507 ,
         0.00084322,  0.02895373],
       [ 0.00474985,  0.00292952, -0.00800761, ..., -0.0361429 ,
         0.00082702,  0.02896362],
       ...,
       [ 0.00470163,  0.00324716, -0.00783575, ..., -0.03603504,
         0.00084905,  0.02891161],
       [ 0.00469569,  0.00323683, -0.00785285, ..., -0.03603852,
         0.00084916,  0.02891328],
       [ 0.00469079,  0.00322541, -0.0078637 , ..., -0.03604023,
         0.00085111,  0.02891425]])

In [19]:
data_1 = np.load(os.path.join(corrsdir, 'lev_1_rest.npy'))

In [20]:
data_1

array([[0.67791692, 0.69482554, 0.68652665, ..., 0.51751886, 0.52131083,
        0.56299987],
       [0.67062577, 0.68782968, 0.67891135, ..., 0.50852289, 0.51226615,
        0.55458279],
       [0.66328066, 0.68075285, 0.67115112, ..., 0.49944956, 0.50336628,
        0.54605204],
       ...,
       [0.64311738, 0.66809795, 0.67043827, ..., 0.51054083, 0.47364042,
        0.55140913],
       [0.65128266, 0.67576094, 0.67812503, ..., 0.51882848, 0.48392795,
        0.55976356],
       [0.65940173, 0.68335293, 0.68574443, ..., 0.527367  , 0.49420536,
        0.5680266 ]])

In [21]:
data_2 = np.load(os.path.join(corrsdir, 'lev_2_rest.npy'))

In [22]:
data_2

array([[0.72958944, 0.73361583, 0.73575896, ..., 0.59141414, 0.64253216,
        0.58702491],
       [0.72226216, 0.72648828, 0.72870467, ..., 0.5814949 , 0.63364393,
        0.57788729],
       [0.71487135, 0.71926916, 0.72163213, ..., 0.57165385, 0.62468238,
        0.56883825],
       ...,
       [0.72768249, 0.72833249, 0.7266187 , ..., 0.59283252, 0.6746103 ,
        0.56274283],
       [0.73447836, 0.73511093, 0.73352724, ..., 0.60183649, 0.68140581,
        0.57219168],
       [0.74118228, 0.74180691, 0.74033674, ..., 0.61082572, 0.68831013,
        0.58166346]])

## Cluster code to calculate correlations for each order - up to 4th order

In [19]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce

factors = 100
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']

data = []
conds = []


for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))


all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 50
wp = 'laplace'

#results_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results/'

results_dir = '/Users/lucyowen/Desktop/timecorr_env/timecorr_paper/pieman/results/'

corrsdir = os.path.join(results_dir, wp + '_' + str(50) + '_orderedup_corrs')

if not os.path.exists(corrsdir):
    os.makedirs(corrsdir)
            
laplace = {'name': 'Laplace', 'weights': tc.laplace_weights, 'params': {'scale': width}}
delta = {'name': '$\delta$', 'weights': tc.eye_weights, 'params': tc.eye_params}
gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}
mexican_hat = {'name': 'Mexican hat', 'weights': tc.mexican_hat_weights, 'params': {'sigma': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2


for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        if os.path.exists(os.path.join(corrsdir, 'd_1_'+ c +'_'+str(i+1)+'.npy')):
            data = np.load(os.path.join(corrsdir, 'd_1_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_1, axis=2)), axis=2))
            
            data = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=,
                                                         weights_function=weights_fun, weights_params=weights_params))
            
            np.save(os.path.join(corrsdir, 'd_1_'+c+'_'+str(i+1)+'.npy'), data_1)
            np.save(os.path.join(corrsdir, 'lev_1_'+c+'_'+str(i+1)+'.npy'), corr)
            
        if os.path.exists(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy')):
            data_r = np.load(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_r = np.asarray(reduce([x for x in data_1], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy'), data_1_r)
            
        del data
        
        if os.path.exists(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy')):
            data_2 = np.load(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_2 = np.asarray(tc.timecorr([x for x in data_1_r], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_2, axis=2)), axis=2))
            np.save(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy'), data_2)
            np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)
        if os.path.exists(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy')):
            data_2_r = np.load(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_2_r = np.asarray(reduce([x for x in data_2], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy'), data_2_r)
        del data_2
        if os.path.exists(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy')):
            data_3 = np.load(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_3 = np.asarray(tc.timecorr([x for x in data_2_r], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_3, axis=2)), axis=2))
            np.save(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy'), data_3)
            np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)
        if os.path.exists(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy')):
            data_3_r = np.load(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_3_r = np.asarray(reduce([x for x in data_3], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy'), data_3_r)
        del data_3
        lev = np.asarray(tc.timecorr([x for x in data_3_r], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_4_'+c+'_'+str(i+1)+'.npy'), corr)
        del lev

IndexError: invalid index to scalar variable.

In [3]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce


In [4]:
data_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results'
corrs_dir = os.path.join(data_dir, 'laplace_50_corrs')

## For sliced correlations:

In [ ]:
levels = [1,2, 3, 4]
groups = [1, 2]
conditions = ['intact', 'paragraph', 'word', 'rest']

n = .1
percentages = np.arange(n, 1 + n, n)
for l in levels:
    for c in conditions:
        for g in groups:
            con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ f'_{g}.npy')
            try:
                corrs = np.load(con)
                mat_corrs = tc.helpers.vec2mat(corrs)
                sliced_corrs = np.array([])
                next_corrdir = os.path.join(data_dir, 'sliced_corrs', f'level_{l}', f'group {g}')
                if not os.path.exists(next_corrdir):
                    os.makedirs(next_corrdir)
                for p in percentages:
                    s = int(p*np.shape(mat_corrs)[-1])
                    slice_corr = mat_corrs[:, :, s-1]
                    if sliced_corrs.shape[0] == 0:
                        sliced_corrs = slice_corr
                    else:
                        sliced_corrs = np.dstack((sliced_corrs, slice_corr))
                np.save(os.path.join(next_corrdir, f'{c}.npy'), sliced_corrs)
            except:
                print('issue loading: ' + con)
                pass

## For averaged correlations:

In [ ]:
levels = [1,2, 3, 4]
groups = [1, 2]
conditions = ['intact', 'paragraph', 'word', 'rest']


for l in levels:
    for c in conditions:
        for g in groups:
            con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ f'_{g}.npy')
            try:
                corrs = np.load(con)
                mat_corrs = tc.helpers.vec2mat(corrs)

                next_corrdir = os.path.join(data_dir, 'mean_corrs', f'level_{l}', f'group {g}')
                if not os.path.exists(next_corrdir):
                    os.makedirs(next_corrdir)

                mean_corrs = mat_corrs.mean(axis=2)

                np.save(os.path.join(next_corrdir, f'{c}.npy'), mean_corrs)
            
            except:
                print('issue loading: ' + con)
                pass